### Libraries

In [1]:
import pickle
import os

#basic
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

#tensorflow and keras
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Flatten, MaxPooling2D, Dropout, Resizing, Rescaling, RandomBrightness, RandomContrast, RandomCrop, RandomFlip, RandomRotation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import Model
from keras.utils import load_img, img_to_array

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

#open cv
import cv2 as cv


from cascid.configs import config, pad_ufes
from cascid import database

# Local py script
#from model import *

### Globals

In [2]:
FERNANDO_PATH = config.DATA_DIR / 'experiments' / 'fernando'
FERNANDO_PATH.mkdir(exist_ok=True, parents=True)

IMAGE_CACHE = FERNANDO_PATH / 'img_cache.pkl'
FEATURES_FILE = FERNANDO_PATH / 'features.pkl'
MODEL_PATH = FERNANDO_PATH / 'models' / 'deep_learning'
IMDIR = pad_ufes.IMAGES_DIR # Can also be pad_ufes.IMAGES_DIR 

In [3]:
RANDOM_STATE = 42
TRAIN_SIZE = 0.7
VALIDATION_SIZE = 0.15
TEST_SIZE = 0.15
EPOCHS = 3000
IMAGE_SHAPE = (64, 64, 3)
BATCH_SIZE = 16

### Load images

In [4]:
def load_image(name: str):
    pil_img = load_img(
        str(IMDIR / name),
        grayscale=False,
        color_mode='rgb',
        target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
        interpolation='nearest',
        keep_aspect_ratio=False
    )

    return img_to_array(pil_img, dtype=np.uint8)

In [5]:
df = database.get_db()

In [6]:
MulticlassEncoder = OneHotEncoder(sparse=False) # OHE for y encoding
Y = MulticlassEncoder.fit_transform(df[["diagnostic"]].to_numpy())
x_train, x_test, y_train, y_test = train_test_split(df["img_id"].to_numpy(), Y, test_size=0.2, random_state=RANDOM_STATE)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=RANDOM_STATE)

In [7]:
reader = lambda img_path_list : np.array(list(map(load_image, img_path_list)))

In [8]:
image_dict = {
        "train": reader(x_train),
        "test": reader(x_test),
        "valid": reader(x_valid)
    }

In [9]:
# Write image cache
with open(IMAGE_CACHE, 'wb') as file:
    pickle.dump(image_dict, file)
print("Read operations done, cache file available at {}".format(IMAGE_CACHE))

Read operations done, cache file available at /home/fernandofincatti/.cascid_data/experiments/fernando/img_cache.pkl


In [10]:
# Return to original variables
x_train = image_dict["train"]
x_test = image_dict["test"]
x_valid = image_dict["valid"]

### Model

In [11]:
input_layer = keras.Sequential([
    Rescaling(1./255), # Rescale from 0 to 255 UINT8 to 0 to 1 float.
])

2022-09-16 18:04:27.391785: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:04:27.396885: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:04:27.397036: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-16 18:04:27.398060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [12]:
augmentor = keras.Sequential([
    RandomBrightness(factor=(-0.3, 0.3), value_range=(0.0, 1.0), seed=RANDOM_STATE), # Randomly change brightness anywhere from -30% to +30%
    RandomContrast(factor=0.5, seed=RANDOM_STATE), # Randomly change contrast anywhere from -30% to +30%
    RandomFlip(mode="horizontal_and_vertical", seed=RANDOM_STATE), # Randomly flip images either horizontally, vertically or both
    RandomRotation(factor=(-0.3, 0.3), fill_mode="nearest", interpolation="bilinear", seed=RANDOM_STATE), # Randomly rotate anywhere from -30% * 2PI to +30% * 2PI, filling gaps by using 'nearest' strategy
])

In [13]:
resnet = keras.applications.ResNet50(
    weights='imagenet',
    input_shape=IMAGE_SHAPE,
    pooling='avg',
    include_top=False
)

In [14]:
resnet.trainable = False  #to make sure it's not being trained
# Augmentation only on training
feature_extractor_train = keras.Sequential([
    input_layer,
    augmentor,
    resnet
])

In [15]:
# Test/Validation only get rescaled
feature_extractor_test_valid = keras.Sequential([
    input_layer,
    resnet
])

In [16]:
features_train = feature_extractor_train(x_train)
features_valid = feature_extractor_test_valid(x_valid)
features_test = feature_extractor_test_valid(x_test)

2022-09-16 18:04:30.808045: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


In [17]:
features = {
        "train": features_train.numpy(),
        "test": features_test.numpy(),
        "valid": features_valid.numpy(),
        "y_train": y_train,
        "y_test": y_test,
        "y_valid": y_valid,
    }

In [18]:
with open(FEATURES_FILE, 'wb') as file:
        pickle.dump(features, file)

### Training

In [19]:
with open(FEATURES_FILE, 'rb') as file:
        features = pickle.load(file)

In [20]:
x_train = features["train"]
x_test = features["test"]
x_valid = features["valid"]
y_train = features["y_train"]
y_test = features["y_test"]
y_valid = features["y_valid"]

In [21]:
model = keras.Sequential([
    Input(shape = features["train"].shape[1]),
    Dense(128, activation='relu'),
    Dropout(0.1),
    Dense(128),
    Dropout(0.1),
    Dense(64),
    Dropout(0.1),
    Dense(y_train.shape[1], activation='softmax')
])

In [22]:
model.compile(optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [23]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    patience=100,
    restore_best_weights=True
)

In [24]:
training_history = model.fit(
    features["train"],
    y_train,
    epochs=EPOCHS,
    validation_split=0.2,
    batch_size=BATCH_SIZE,
    callbacks=[early_stopping]
)


Epoch 1/3000
74/74 [==============================] - 1s 3ms/step - loss: 1.5674 - accuracy: 0.3427 - val_loss: 1.4848 - val_accuracy: 0.3605
Epoch 2/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.5168 - accuracy: 0.3520 - val_loss: 1.4825 - val_accuracy: 0.3605
Epoch 3/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.5261 - accuracy: 0.3529 - val_loss: 1.4885 - val_accuracy: 0.3435
Epoch 4/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.5138 - accuracy: 0.3622 - val_loss: 1.5681 - val_accuracy: 0.3435
Epoch 5/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.5128 - accuracy: 0.3750 - val_loss: 1.4744 - val_accuracy: 0.3435
Epoch 6/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.4921 - accuracy: 0.3605 - val_loss: 1.4736 - val_accuracy: 0.3435
Epoch 7/3000
74/74 [==============================] - 0s 2ms/step - loss: 1.4890 - accuracy: 0.3759 - val_loss: 1.4703 - val_accuracy: 0.3435
Epoch 

In [25]:
model.save(MODEL_PATH)

INFO:tensorflow:Assets written to: /home/fernandofincatti/.cascid_data/experiments/fernando/models/deep_learning/assets


In [26]:
with open(MODEL_PATH / 'history.pkl', 'wb') as fl:
    pickle.dump(training_history.history, fl)
training_history = training_history.history

In [27]:
model.evaluate(x=x_test, y=y_test)

15/15 [==============================] - 0s 1ms/step - loss: 1.5619 - accuracy: 0.2891


[1.5619410276412964, 0.28913044929504395]

15/15 [==============================] - 0s 785us/step


array([[0.39797878, 0.26922172, 0.0266172 , 0.09026791, 0.0829881 ,
        0.13292632],
       [0.39595297, 0.26885542, 0.02708264, 0.09102   , 0.08358371,
        0.13350527],
       [0.39716727, 0.26910025, 0.02678728, 0.09055118, 0.08323975,
        0.13315426],
       ...,
       [0.39802122, 0.26923916, 0.02661093, 0.09024911, 0.08298463,
        0.13289487],
       [0.4035434 , 0.2702216 , 0.02540503, 0.08821656, 0.08140206,
        0.13121133],
       [0.39934292, 0.26944998, 0.02632866, 0.08977287, 0.08260016,
        0.13250537]], dtype=float32)

In [35]:
pd.Series([np.argmax(l) for l in model.predict(x_test)]).value_counts(True)

1    0.368027
0    0.324490
5    0.108844
3    0.104762
4    0.073469
2    0.020408
dtype: float64